### Getting Started with Local Large Language Models

In this file, a local language model will be applied to create models of small-scale multibody problems. 

First, the libraries are loaded:  
* gpt4all: used to run models locally; can be used both as a GUI and a Python library
* transformers: library for pre-trained models for inference applications and fine-tuning
* Hugging Face Hub: is a git-based repository, offering a wide range of pre-trained models

If enough VRAM (GPU) or RAM (CPU) is available, __Phi-4__ can be used. The 4-bit quantized model (Q4) requires 8 GB of memory. In case your computer does not have enough memory __Phi-3__ can also be used - although it is not as capable - by setting  
__<code>flagSmall = True </code>__  
in the next code block. Note that the respective model is automatically downloaded when running this script. You can technically run models that exceed the available memory by swapping to the disk, but this will slow them down considerably. 

Task: 
* Run script locally
* Optional: browse models on [huggingface](https://huggingface.co/) and find a model you want to run



In [6]:
from gpt4all import GPT4All
from transformers import AutoConfig
from huggingface_hub import hf_hub_download
from LLMHelperFunctions import CheckOutputLLM # helper function
import torch # only used to check if cuda is available
import time

flagSmall = False

if flagSmall: 
    # approx 2.4GB
    repo_id = 'microsoft/phi-3-mini-4k-instruct-gguf'
    filename =  "Phi-3-mini-4k-instruct-q4.gguf"
else: 
    repo_id = "bartowski/phi-4-gguf"
    filename = "phi-4-Q4_K_S.gguf"

### Inspect model

We can now inspect the model configuration from the repository. Note that, depending on the model, there might be several versions in the repository available, see e.g. [phi-4](https://huggingface.co/microsoft/phi-4). 



In [7]:
config = AutoConfig.from_pretrained(repo_id)
print('repo id: ', repo_id, config, '\n')

repo id:  bartowski/phi-4-gguf BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_type": "bart",
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": false,
  "transformers_version": "4.53.1",
  "use_cache": true,
  "vocab_size": 50265
}
 



### Load Model


The model is automatically downloaded and then loaded either on the GPU ('cuda') if available or into the RAM for running inference on the CPU.  
By default, the model location is:  
<!-- <code> `C:\Users\<username>\.cache\huggingface\` </code> -->
<code> `%USERPROFILE%\.cache\huggingface\hub` </code>

__Note__: If you are short on memory on your PC, you should later delete the LLM file model manually as it will be not deleted with the Python environment and the models are multiple GBs. 

In [8]:
modelPath = hf_hub_download(repo_id=repo_id, filename=filename)
print('model {} downloaded to local directory: \n{}\n'.format(filename, modelPath))
try: 
    model = GPT4All(modelPath, device='cuda')
    print('running LLM on GPU')
except: 
    model = GPT4All(modelPath, device='cpu')
    print('running LLM on CPU')

model phi-4-Q4_K_S.gguf downloaded to local directory: 
C:\Users\C8501100\.cache\huggingface\hub\models--bartowski--phi-4-gguf\snapshots\19cd65f97c2f1712a81c506611d3f9c94b16a1e1\phi-4-Q4_K_S.gguf

running LLM on GPU


In [9]:
strQuestion = "How many eigenmodes does a two-mass spring-damper have?"
print(strQuestion)
t1 = time.time()
output = model.generate(strQuestion, max_tokens=int(1e3))
dt = time.time() - t1
print(f'inference took {round(dt, 2)}s')
print(output)


How many eigenmodes does a two-mass spring-damper have?
inference took 4.39s
**

To determine the number of eigenmodes in a system, we need to consider its degrees of freedom. A two-mass spring-damper system typically consists of:

- Two masses
- Springs connecting these masses and possibly fixed points (e.g., walls)
- Dampers providing damping forces

Each mass can move independently along one dimension (assuming motion is constrained in a single direction, such as horizontally). Therefore, the system has two degrees of freedom.

The number of eigenmodes corresponds to the number of independent ways the system can oscillate. For each degree of freedom, there is typically an associated mode shape and natural frequency. Thus, for this two-mass spring-damper system with two degrees of freedom, it will have **two eigenmodes**.

These modes describe how the masses move relative to one another when the system vibrates naturally (without external forcing). Each mode has a specific pattern of

### Output

The model is trained to predict the next tokens, but does not neccesarily stop after answering the question. There are special tokens to help structure inputs and outputs, see [4-3_CreateOscillator](4-3_CreateOscillator.ipynb). 

